In [1]:
# Idea: permute team members to make model more robust

In [2]:
import pandas as pd

In [3]:
from itertools import permutations

In [16]:
complete_permutations = []
for p in permutations(range(1, 6)):
    unique = True
    for i in range(5):
        if p[i] == i + 1:
            unique = False
            break
    for c in complete_permutations:
        for i in range(5):
            if p[i] == c[i]:
                unique = False
                break
        if not unique:
            break
    if unique:
        complete_permutations.append(p)
complete_permutations

[(2, 1, 4, 5, 3), (3, 4, 5, 1, 2), (4, 5, 2, 3, 1), (5, 3, 1, 2, 4)]

In [17]:
train_df = pd.read_csv('data/train_features.csv', index_col=0)

In [52]:
perm_radiant, perm_dire = complete_permutations[:2]

In [53]:
X = train_df.copy()
new_r_team = []
new_d_team = []
for i, prad, pdire in zip(range(1, 6), perm_radiant, perm_dire):
    p = X[[c for c in X.columns if c.startswith('r'+str(prad))]]
    p.columns = [c[0] + str(i) + c[2:] for c in p.columns]
    new_r_team.append(p)
    
    p = X[[c for c in X.columns if c.startswith('d'+str(pdire))]]
    p.columns = [c[0] + str(i) + c[2:] for c in p.columns]
    new_d_team.append(p)

In [42]:
general_features = train_df.iloc[:, :5]

In [61]:
enh_train = []

combined_df = pd.concat([general_features] + new_r_team + new_d_team, axis=1)
enh_train.append(combined_df)


In [63]:
enh_train.append(combined_df)

In [13]:
len(complete_permutations)

4

In [65]:
for perm_rad, perm_dire in zip(complete_permutations, complete_permutations[1:] + [complete_permutations[0]]):
    print(perm_rad, perm_dire)

(2, 1, 4, 5, 3) (3, 4, 5, 1, 2)
(3, 4, 5, 1, 2) (4, 5, 2, 3, 1)
(4, 5, 2, 3, 1) (5, 3, 1, 2, 4)
(5, 3, 1, 2, 4) (2, 1, 4, 5, 3)


In [66]:
import gc

In [67]:
gc.collect()

4763

In [84]:
import re

In [85]:
X.columns

Index(['game_time', 'game_mode', 'lobby_type', 'objectives_len', 'chat_len',
       'r1_hero_id', 'r1_kills', 'r1_deaths', 'r1_assists', 'r1_denies',
       ...
       'd5_stuns', 'd5_creeps_stacked', 'd5_camps_stacked', 'd5_rune_pickups',
       'd5_firstblood_claimed', 'd5_teamfight_participation',
       'd5_towers_killed', 'd5_roshans_killed', 'd5_obs_placed',
       'd5_sen_placed'],
      dtype='object', length=245)

In [90]:
hero_features = [c for c in X.columns if re.search(r'[rd][0-9]_.*', c) ]

In [76]:
X = train_df.copy()
enh_train = []

for perm_rad, perm_dire in zip(complete_permutations, complete_permutations[1:] + [complete_permutations[0]]):
    new_r_team = []
    new_d_team = []
    for i, prad, pdire in zip(range(1, 6), perm_rad, perm_dire):
        p = X[[c for c in X.columns if c.startswith('r'+str(prad))]]
        p.columns = [c[0] + str(i) + c[2:] for c in p.columns]
        new_r_team.append(p)

        p = X[[c for c in X.columns if c.startswith('d'+str(pdire))]]
        p.columns = [c[0] + str(i) + c[2:] for c in p.columns]
        new_d_team.append(p)
    combined_df = pd.concat([general_features] + new_r_team + new_d_team, axis=1)
    enh_train.append(combined_df)
    gc.collect()
X_enh = pd.concat([X] + enh_train)
X_enh.shape

(198375, 245)

In [70]:
train_targets = pd.read_csv('data/train_targets.csv', index_col=0)

In [71]:
y = train_targets.radiant_win

In [77]:
y_enh = pd.concat([y] * (1 + len(enh_train)))

In [80]:
y_enh = pd.DataFrame(y_enh)

In [79]:
X_enh.to_csv('perm_X.csv')

/Volumes/Storage/goinfre/ptyshevs/env/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [82]:
y_enh.to_csv('perm_y.csv')

In [92]:
def permute_players(X, Y):
    hero_features = [c for c in X.columns if re.search(r'[rd][0-9]_.*', c)]
    permutations = [(2, 1, 4, 5, 3), (3, 4, 5, 1, 2), (4, 5, 2, 3, 1), (5, 3, 1, 2, 4)]
    enh_train = []

    for perm_rad, perm_dire in zip(permutations, permutations[1:] + [permutations[0]]):
        new_r_team = []
        new_d_team = []
        for i, prad, pdire in zip(range(1, 6), perm_rad, perm_dire):
            p = X[[c for c in hero_features if c.startswith('r'+str(prad))]]
            p.columns = [c[0] + str(i) + c[2:] for c in p.columns]
            new_r_team.append(p)

            p = X[[c for c in hero_features if c.startswith('d'+str(pdire))]]
            p.columns = [c[0] + str(i) + c[2:] for c in p.columns]
            new_d_team.append(p)
        combined_df = pd.concat([general_features] + new_r_team + new_d_team, axis=1)
        enh_train.append(combined_df)
        gc.collect()
    X_enh = pd.concat([X] + enh_train)
    y_enh = pd.DataFrame(pd.concat([Y.radiant_win] * (1 + len(enh_train))), columns=['radiant_win'])
    return X_enh, y_enh

In [94]:
try_X, try_Y = permute_players(X, train_targets)

In [96]:
try_X.shape, try_Y.shape

((198375, 245), (198375, 1))